# Saving Schema Data to Vector Database

My initial thought was to build a model that checks for similarity between the prompt and the schema information. That may still be part of the long-term roadmap, but in doing research I found I could get acceptable results in a much simpler way using a vector database through langchain. By savings documents related to our schemas to this database, we can then query based on the question and return the documents that are most closely related. With this, we'll try to all be bundled into langchain. woohoo!

Big shoutout to this great blogpost that provided some of the framework: https://canvasapp.com/blog/text-to-sql-in-production

In [1]:
from watermark import watermark
print(watermark())

Last updated: 2023-08-02T07:22:45.222238-07:00

Python implementation: CPython
Python version       : 3.11.4
IPython version      : 8.14.0

Compiler    : Clang 15.0.7 
OS          : Darwin
Release     : 22.5.0
Machine     : x86_64
Processor   : i386
CPU cores   : 8
Architecture: 64bit



In [11]:
import pandas as pd
import json
import os

from dotenv import load_dotenv

from langchain import SQLDatabase
from langchain.docstore.document import Document
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

from sqlalchemy import exc

## Current Method

In this first launch, I settled on saving the schema, table, column list, DDL, and 3 sample rows into each document.
There is also metadata attached to each document with the schema, table, and column list to easily pull from the query results.

## Establish Vector Database

There are a few options for databases, but I'll go with Chroma because it is open source, makes local device use "easy", and has built-in connections with langchain.

A key component in running apps using langchain is the ability store and work with embeddings, which is how AI models natively represent data of all kinds. Langchain will provide the application framework and Chroma will provide the vector store.

Within that we can also do some information retrieval from the database, finding the most relevant tables based on the user question. This will allow us to engineer a better prompt to feed to our gpt chatbot.

In [12]:
#setup embeddings using HuggingFace
embeddings  = HuggingFaceEmbeddings()

### Create Functions

In [13]:
def get_json(path):
    """Return json file from specified filepath"""
    with open(path, "r") as f:
        json_file = json.load(f)
    
    return json_file

In [14]:
#point to where our .sqlite files are saved
db_path = '../data/processed/db/'

In [15]:
def connect_db(db_path, target_schema):
    """
    Take in the identified schema and connect to the sqlite database with that name
    """
    db_filepath = db_path
    db_filename = target_schema + '.sqlite'

    #point to database
    base_dir = os.path.dirname(os.path.abspath(db_filepath+db_filename)) #get the full path within the device
    db_path = os.path.join(base_dir, db_filename) #combine with filename to get db_path
    db = SQLDatabase.from_uri("sqlite:///" + db_path) #connect via the lanchain method

    return db

In [16]:
#define new document builder
def prep_chroma_documents_v2(json_path):
    """Take json file and work through it to prepare for load to Chroma.
    Instead of list comprehension, establis the blank list and loop through the json.
    Then saves to the list using the langchain docstore.document -> Document module.

    This version - adding the table info using the langchain SQLDatabase SQLAlchemy wrapper to get table info to add to metadata.
    Would like to not reconnect to the database each time, but instead connect to each schema once and then loop through the tables. But I think this will be easier for now, even if it's less efficient.

    This works specifically with the content and metadata we want for this project"""
    docs = []
    for item in get_json(json_path):
        #connect to database
        db = connect_db(db_path=db_path, target_schema=item['schema'])

        #create variables
        schema = item['schema']
        table = item['table']
        columns = json.dumps([col['c_name'] for col in item['columns']])
        try:
            table_info = db.get_table_info_no_throw(table_names=[table]) #put try-except here becasue there are some issues in the source sqlite database. I want to call this out, but continue.
        except exc.SQLAlchemyError as e:
            table_info = ""
            print(schema + "-" + table + ": " + str(e))
        except TypeError as te:
            print(schema + "-" + table + ": " + str(te))          

        #create document
        doc = Document(
            page_content=f"""
Schema: {schema}
Table: {table}
Columns: {columns}
DDL:
    {table_info}
""",
            metadata={
                'schema': schema,
                'table': table,
                'columns': columns,
            }
        )
        docs.append(doc)
    
    return docs

In [17]:
table_docs = prep_chroma_documents_v2('../data/interim/schema_info.json')

/Users/brettly/opt/anaconda3/envs/text2sql/lib/python3.11/site-packages/langchain/sql_database.py:111: SAWarning: Could not instantiate type <class 'sqlalchemy.sql.sqltypes.INTEGER'> with reflected arguments ['11']; using no arguments.
  self._metadata.reflect(


baseball_1-all_star: Could not initialize target column for ForeignKey 'player.team_id' on table 'fielding_postseason': table 'player' has no column named 'team_id'
baseball_1-appearances: Could not initialize target column for ForeignKey 'player.team_id' on table 'fielding_postseason': table 'player' has no column named 'team_id'
baseball_1-batting: Could not initialize target column for ForeignKey 'player.team_id' on table 'fielding_postseason': table 'player' has no column named 'team_id'
baseball_1-batting_postseason: Could not initialize target column for ForeignKey 'player.team_id' on table 'fielding_postseason': table 'player' has no column named 'team_id'
baseball_1-college: Could not initialize target column for ForeignKey 'player.team_id' on table 'fielding_postseason': table 'player' has no column named 'team_id'
baseball_1-fielding: Could not initialize target column for ForeignKey 'player.team_id' on table 'fielding_postseason': table 'player' has no column named 'team_id'

/Users/brettly/opt/anaconda3/envs/text2sql/lib/python3.11/site-packages/langchain/sql_database.py:111: SAWarning: WARNING: SQL-parsed foreign key constraint '('Cust_ID', 'customer', 'Cust_ID')' could not be located in PRAGMA foreign_keys for table loan
  self._metadata.reflect(


restaurants-GEOGRAPHIC: Could not initialize target column for ForeignKey 'RESTAURANT.RESTAURANT_ID' on table 'LOCATION': table 'RESTAURANT' has no column named 'RESTAURANT_ID'
restaurants-LOCATION: Could not initialize target column for ForeignKey 'RESTAURANT.RESTAURANT_ID' on table 'LOCATION': table 'RESTAURANT' has no column named 'RESTAURANT_ID'
restaurants-RESTAURANT: Could not initialize target column for ForeignKey 'RESTAURANT.RESTAURANT_ID' on table 'LOCATION': table 'RESTAURANT' has no column named 'RESTAURANT_ID'
sakila_1-film: (in table 'film', column 'rating'): Can't generate DDL for NullType(); did you forget to specify a type on this Column?


/Users/brettly/opt/anaconda3/envs/text2sql/lib/python3.11/site-packages/langchain/sql_database.py:111: SAWarning: WARNING: SQL-parsed foreign key constraint '('store_id', 'store', 'store_id')' could not be located in PRAGMA foreign_keys for table staff
  self._metadata.reflect(
/Users/brettly/opt/anaconda3/envs/text2sql/lib/python3.11/site-packages/langchain/sql_database.py:111: SAWarning: WARNING: SQL-parsed foreign key constraint '('Event_ID', 'Events', 'Event_ID')' could not be located in PRAGMA foreign_keys for table Assets_in_Events
  self._metadata.reflect(


store_product-district: Could not initialize target column for ForeignKey 'product.Product_ID' on table 'store_product': table 'product' has no column named 'Product_ID'
store_product-product: Could not initialize target column for ForeignKey 'product.Product_ID' on table 'store_product': table 'product' has no column named 'Product_ID'
store_product-store: Could not initialize target column for ForeignKey 'product.Product_ID' on table 'store_product': table 'product' has no column named 'Product_ID'
store_product-store_district: Could not initialize target column for ForeignKey 'product.Product_ID' on table 'store_product': table 'product' has no column named 'Product_ID'
store_product-store_product: Could not initialize target column for ForeignKey 'product.Product_ID' on table 'store_product': table 'product' has no column named 'Product_ID'
wta_1-matches: fromisoformat: argument must be str
wta_1-players: fromisoformat: argument must be str
wta_1-rankings: fromisoformat: argument m

/Users/brettly/opt/anaconda3/envs/text2sql/lib/python3.11/site-packages/langchain/sql_database.py:111: SAWarning: Could not instantiate type <class 'sqlalchemy.sql.sqltypes.BIGINT'> with reflected arguments ['20']; using no arguments.
  self._metadata.reflect(


### Create VectorDB

In [18]:
#setup directory to store database on disk
persist_dir_new = '../data/processed/chromadb/schema-table-info'

In [19]:
###commenting out since this was already run and created###

#vectordb_new = Chroma.from_documents(documents=table_docs, embedding=embeddings, persist_directory=persist_dir_new)

#vectordb_new.persist() #should only need to run this out of the notebook, not a .py file.
vectordb_new=None

### Test - Table Info Method

In [20]:
# load from disk
vectordb_info = Chroma(persist_directory=persist_dir_new, embedding_function=embeddings)

test = "What is the average number of employees of the departments whose rank is between 10 and 15?" #one of the prompts from the training data

test_docs = vectordb_info.similarity_search(test, k=3)

test_docs

[Document(page_content='\nSchema: hr_1\nTable: departments\nColumns: ["DEPARTMENT_ID", "DEPARTMENT_NAME", "MANAGER_ID", "LOCATION_ID"]\nDDL:\n    \nCREATE TABLE departments (\n\t"DEPARTMENT_ID" DECIMAL(4, 0) DEFAULT \'0\' NOT NULL, \n\t"DEPARTMENT_NAME" VARCHAR(30) NOT NULL, \n\t"MANAGER_ID" DECIMAL(6, 0) DEFAULT NULL, \n\t"LOCATION_ID" DECIMAL(4, 0) DEFAULT NULL, \n\tPRIMARY KEY ("DEPARTMENT_ID")\n)\n\n/*\n3 rows from departments table:\nDEPARTMENT_ID\tDEPARTMENT_NAME\tMANAGER_ID\tLOCATION_ID\n10\tAdministration\t200\t1700\n20\tMarketing\t201\t1800\n30\tPurchasing\t114\t1700\n*/\n', metadata={'schema': 'hr_1', 'table': 'departments', 'columns': '["DEPARTMENT_ID", "DEPARTMENT_NAME", "MANAGER_ID", "LOCATION_ID"]'}),
 Document(page_content='\nSchema: department_management\nTable: department\nColumns: ["Department_ID", "Name", "Creation", "Ranking", "Budget_in_Billions", "Num_Employees"]\nDDL:\n    \nCREATE TABLE department (\n\t"Department_ID" INTEGER, \n\t"Name" TEXT, \n\t"Creation" TEX

In [21]:
#test mmr retriever mentioned in langchain docs (Maximum Marginal Relevance)
retriever = vectordb_info.as_retriever(search_type="mmr")

retriever.get_relevant_documents(test)

[Document(page_content='\nSchema: hr_1\nTable: departments\nColumns: ["DEPARTMENT_ID", "DEPARTMENT_NAME", "MANAGER_ID", "LOCATION_ID"]\nDDL:\n    \nCREATE TABLE departments (\n\t"DEPARTMENT_ID" DECIMAL(4, 0) DEFAULT \'0\' NOT NULL, \n\t"DEPARTMENT_NAME" VARCHAR(30) NOT NULL, \n\t"MANAGER_ID" DECIMAL(6, 0) DEFAULT NULL, \n\t"LOCATION_ID" DECIMAL(4, 0) DEFAULT NULL, \n\tPRIMARY KEY ("DEPARTMENT_ID")\n)\n\n/*\n3 rows from departments table:\nDEPARTMENT_ID\tDEPARTMENT_NAME\tMANAGER_ID\tLOCATION_ID\n10\tAdministration\t200\t1700\n20\tMarketing\t201\t1800\n30\tPurchasing\t114\t1700\n*/\n', metadata={'schema': 'hr_1', 'table': 'departments', 'columns': '["DEPARTMENT_ID", "DEPARTMENT_NAME", "MANAGER_ID", "LOCATION_ID"]'}),
 Document(page_content='\nSchema: insurance_and_eClaims\nTable: Staff\nColumns: ["Staff_ID", "Staff_Details"]\nDDL:\n    \nCREATE TABLE "Staff" (\n\t"Staff_ID" INTEGER NOT NULL, \n\t"Staff_Details" VARCHAR(255) NOT NULL, \n\tPRIMARY KEY ("Staff_ID")\n)\n\n/*\n3 rows from Sta

This didn't put it in the 1st spot, but it is in the top three. After testing on all of the questions, this document structure ended up being the best performing of the three methods I tried. This is despite the fact that some of the table info steps didn't run due to database issues for 3-4 of the tables.

## Other Methods

### Simple Schema-Table

In this first iteration I tried, the document content only had the schema-table combo.

In [22]:
def prep_chroma_documents(json_path):
    """Take json file and work through it to prepare for load to Chroma.
    Instead of list comprehension, establis the blank list and loop through the json.
    Then saves to the list using the langchain docstore.document -> Document modeul

    This works specifically with the content and metadata we want for this project"""
    docs = []
    for item in get_json(json_path):
        doc = Document(
            page_content=f"Schema Table: {item['schema_split'] + ' ' + item['table_split']}",
            metadata={
                'schema': item['schema'],
                'table': item['table'],
                'columns': json.dumps([col['c_name'] for col in item['columns']])
            }
        )
        docs.append(doc)
    
    return docs

In [23]:
table_info = prep_chroma_documents('../data/interim/schema_info.json')

table_info[:3]

[Document(page_content='Schema Table: academic author', metadata={'schema': 'academic', 'table': 'author', 'columns': '["aid", "homepage", "name", "oid"]'}),
 Document(page_content='Schema Table: academic cite', metadata={'schema': 'academic', 'table': 'cite', 'columns': '["cited", "citing"]'}),
 Document(page_content='Schema Table: academic conference', metadata={'schema': 'academic', 'table': 'conference', 'columns': '["cid", "homepage", "name"]'})]

In [24]:
#setup directory to store database on disk
persist_dir = '../data/processed/chromadb/schema-table-split'

In [7]:
###commenting out since we already created this database

#vectordb = Chroma.from_documents(documents=table_info, embedding=embeddings, persist_directory=persist_dir)

#### Persist DB

In [25]:
#vectordb.persist() #think I need to call this mainly because I'm in an ipynb.
vectordb=None

## Test Vector Database Query

In [26]:
# load from disk
vectordb = Chroma(persist_directory=persist_dir, embedding_function=embeddings)

In [27]:
query = "How many heads of the departments are older than 56?" #one of the prompts from the training data

docs = vectordb.similarity_search(query, k=1)

In [28]:
print('Most Likely Schema and Table:\n')
i=0
for doc in docs:
    print(docs[i].metadata['schema'] + ' - ' + docs[i].metadata['table'])
    i+=1

Most Likely Schema and Table:

department_management - head


# Test Entity Extraction

https://canvasapp.com/blog/text-to-sql-in-production

This blog suggests doing some entity extraction from the question. I can try the same method they do of calling to a chat LLM. I could also try using a HuggingFace model specific to text classification or entity extraction.

I could try to train an entity recognition model on the schema information. But first I'll try using a chat model.

In [29]:
from dotenv import load_dotenv

from langchain import HuggingFaceHub
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [30]:
#establish examples of what we want to get back from the model

extract_prompt = """question: How many teams are there?
response: team

question: What user spent the most money in March?
response: user, money

question: What is the name of the instructor who advises the student with the greatest number of total credits?
response: instructor, advisor, student, credits
"""

In [31]:
#create instruction for the LLM

instruction = """
For a given question, determine the keywords for determining what tables should be used to write a SQL query to answer the question.
"""

In [32]:
#create the full prompt content combining the examples and the instruction

prompt = (
    instruction
    + "\nHere are are three question/response examples: "
    + extract_prompt
    + "\n\nquestion: "
    + test
    + "\nresponse:"
)

In [33]:
#structure the prompt in the langchain template

entity_extr_prompt = PromptTemplate(
    input_variables=[],
    template = prompt
)

In [34]:
#establish the LLM - using google flan again

#get api key
load_dotenv()
hf_api_token = os.getenv('hf_token')

#add path to HF repo
repo_id = 'tiiuae/falcon-7b-instruct'

#establish llm model
llm = HuggingFaceHub(repo_id=repo_id, huggingfacehub_api_token=hf_api_token, model_kwargs={"temperature": .005, "max_length": 512})

#create an LLM chain
chain = LLMChain(llm=llm, prompt=entity_extr_prompt, verbose=False)

In [35]:
#test on question
results= chain.predict() #run llm
print(results)

 average, rank, employee, departments


In [37]:
#run the resul throug the similarity search
vectordb_info.similarity_search(results, k=3)

[Document(page_content='\nSchema: department_management\nTable: department\nColumns: ["Department_ID", "Name", "Creation", "Ranking", "Budget_in_Billions", "Num_Employees"]\nDDL:\n    \nCREATE TABLE department (\n\t"Department_ID" INTEGER, \n\t"Name" TEXT, \n\t"Creation" TEXT, \n\t"Ranking" INTEGER, \n\t"Budget_in_Billions" REAL, \n\t"Num_Employees" REAL, \n\tPRIMARY KEY ("Department_ID")\n)\n\n/*\n3 rows from department table:\nDepartment_ID\tName\tCreation\tRanking\tBudget_in_Billions\tNum_Employees\n1\tState\t1789\t1\t9.96\t30266.0\n2\tTreasury\t1789\t2\t11.1\t115897.0\n3\tDefense\t1947\t3\t439.3\t3000000.0\n*/\n', metadata={'schema': 'department_management', 'table': 'department', 'columns': '["Department_ID", "Name", "Creation", "Ranking", "Budget_in_Billions", "Num_Employees"]'}),
 Document(page_content='\nSchema: hr_1\nTable: departments\nColumns: ["DEPARTMENT_ID", "DEPARTMENT_NAME", "MANAGER_ID", "LOCATION_ID"]\nDDL:\n    \nCREATE TABLE departments (\n\t"DEPARTMENT_ID" DECIMAL(

This single test is promising as it's the first to return our target schema as the primary result. I'm worried about how to efficiently test this, since each run requires a call to an LLM. Maybe I can go to my validation page and just test on a sample of the questions.

### Follow-Up:

I was able to test on 50 of the random questions - only getting 40% of them correctly matched. Trying to go for a larger sample size ran into the hourly limits of Hugging Face Inference. Due to the poor performance on the small sample size and inability to test more, I won't go with this method.

## Final Test -> Remove Table Info from Docs

This will just make the docs have the schema - table - columns to see if that improves the overall results. If not, I'll go with my v2.

In [38]:
#define new document builder
def prep_chroma_documents_v3(json_path):
    """Take json file and work through it to prepare for load to Chroma.
    Instead of list comprehension, establis the blank list and loop through the json.
    Then saves to the list using the langchain docstore.document -> Document module.

    This version - adding the table info using the langchain SQLDatabase SQLAlchemy wrapper to get table info to add to metadata.
    Would like to not reconnect to the database each time, but instead connect to each schema once and then loop through the tables. But I think this will be easier for now, even if it's less efficient.

    This works specifically with the content and metadata we want for this project"""
    docs = []
    for item in get_json(json_path):
        #create variables
        schema = item['schema']
        table = item['table']
        columns = json.dumps([col['c_name'] for col in item['columns']])      

        #create document
        doc = Document(
            page_content=f"""
Schema: {schema}
Table: {table}
Columns: {columns}
""",
            metadata={
                'schema': schema,
                'table': table,
                'columns': columns,
            }
        )
        docs.append(doc)
    
    return docs

In [39]:
#create new docs
new_docs = prep_chroma_documents_v3('../data/interim/schema_info.json')

In [40]:
new_docs[:3]

[Document(page_content='\nSchema: academic\nTable: author\nColumns: ["aid", "homepage", "name", "oid"]\n', metadata={'schema': 'academic', 'table': 'author', 'columns': '["aid", "homepage", "name", "oid"]'}),
 Document(page_content='\nSchema: academic\nTable: cite\nColumns: ["cited", "citing"]\n', metadata={'schema': 'academic', 'table': 'cite', 'columns': '["cited", "citing"]'}),
 Document(page_content='\nSchema: academic\nTable: conference\nColumns: ["cid", "homepage", "name"]\n', metadata={'schema': 'academic', 'table': 'conference', 'columns': '["cid", "homepage", "name"]'})]

In [41]:
#setup directory to store database on disk
persist_dir_3 = '../data/processed/chromadb/sch-tab-col'

In [43]:
###commenting out as this was already created

#vectordb_3 = Chroma.from_documents(documents=new_docs, embedding=embeddings, persist_directory=persist_dir_3)

#vectordb_3.persist() #think I need to call this mainly because I'm in an ipynb.
vectordb_3=None

#### Test New DB

In [44]:
# load from disk
vectordb_3 = Chroma(persist_directory=persist_dir_3, embedding_function=embeddings)

test = "What is the average number of employees of the departments whose rank is between 10 and 15?" #one of the prompts from the training data

test_docs = vectordb_3.similarity_search(test, k=3)

test_docs

[Document(page_content='\nSchema: department_management\nTable: department\nColumns: ["Department_ID", "Name", "Creation", "Ranking", "Budget_in_Billions", "Num_Employees"]\n', metadata={'schema': 'department_management', 'table': 'department', 'columns': '["Department_ID", "Name", "Creation", "Ranking", "Budget_in_Billions", "Num_Employees"]'}),
 Document(page_content='\nSchema: department_store\nTable: Staff_Department_Assignments\nColumns: ["staff_id", "department_id", "date_assigned_from", "job_title_code", "date_assigned_to"]\n', metadata={'schema': 'department_store', 'table': 'Staff_Department_Assignments', 'columns': '["staff_id", "department_id", "date_assigned_from", "job_title_code", "date_assigned_to"]'}),
 Document(page_content='\nSchema: department_management\nTable: management\nColumns: ["department_ID", "head_ID", "temporary_acting"]\n', metadata={'schema': 'department_management', 'table': 'management', 'columns': '["department_ID", "head_ID", "temporary_acting"]'})]

**Observations:**
Looks good on the test, I'll go to my validation notebook and try it on all of the questions.

### Follow-up:

I did the test on the full question list and it returned slightly worse results than the db with the table info. I'll stick with that one.